<a href="https://colab.research.google.com/github/firmai/random-assets/blob/master/Copy_of_FFOOD_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install shap

In [0]:
import pandas as pd 
import shap
import lightgbm as lgb
import numpy as np
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
import numpy as np

X_pr = pd.read_csv("https://github.com/firmai/random-assets/blob/master/data_clean.csv?raw=true").set_index("Unnamed: 0", drop=True)

In [0]:
X_pr.to_csv("frame.csv")

In [0]:
#X_pr = pd.read_csv()

In [58]:
from keras.models import Sequential
from keras.layers import Dense
import numpy as np

Using TensorFlow backend.


In [0]:
X_pr.index.name = 'index'
target = "price"
first = X_pr.sample(int(len(X_pr)/2))
second = X_pr[~X_pr.isin(first)].dropna()


def modelling(first, second, target, inter_portion=0.5, sqr_portion=0.5, contribution_portion=0.9, final_contribution=0.9, final=False):
  
  cols_drop = [target ]

  def shap_frame(first,second, target):

    #could also use linear booster XGBoost
    d_train = lgb.Dataset(first.drop(columns=[target]), label=first[target])
    d_valid = lgb.Dataset(second.drop(columns=[target]), label=second[target])
    params = {
      
        'boosting_type': 'gbdt',
        'objective': 'regression',
        'metric': 'rmsle',
        'max_depth': 6, 
        'learning_rate': 0.1,
        'verbose': 0,
      'num_threads':16}
    n_estimators = 100

    model = lgb.train(params, d_train, 100, verbose_eval=1)

    explainer = shap.TreeExplainer(model)
    shap_values = explainer.shap_values(first.drop([target], axis=1))
    shap_fram = pd.DataFrame(shap_values[:,:], columns=list(first.drop([target], axis=1).columns))
    shap_new = shap_fram.sum().sort_values().to_frame()
    
    return shap_new, explainer, model

  def shap_frame_keras(first,second, target):
    # Set the input shape
    input_shape = (len(first.columns),)
    print(f'Feature shape: {input_shape}')

    # Create the model
    model = Sequential()
    model.add(Dense(16, input_shape=input_shape, activation='relu'))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(1, activation='linear'))


    # Configure the model and start training
    model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_squared_error'])
    model.fit(first.drop(columns=[target]), first[target], epochs=10, batch_size=10, verbose=1, validation_split=0.2)

    explainer = shap.DeepExplainer(model)

    shap_values = explainer.shap_values(first.drop([target], axis=1))
    shap_fram = pd.DataFrame(shap_values[:,:], columns=list(first.drop([target], axis=1).columns))
    shap_new = shap_fram.sum().sort_values().to_frame()
    
    return shap_new, explainer, model


  ##new_df = first.drop(cols_drop, axis=1)

  ### Main Features

  ### Main Calculations
  ##shap_new, explainer, model = shap_frame(first,second,target)

  shap_new, explainer, model = shap_frame_keras(first,second,target)

  shap_new_abs = shap_new[0].abs()
  shap_new_abs = shap_new_abs.sort_values(ascending=False)
  main_ft = shap_new_abs[shap_new_abs.cumsum().sub((shap_new_abs.sum()*sqr_portion)).le(0)]

  preds = model.predict(second.drop(columns=[target]))
  mse = mean_squared_error(second[target], preds)
  print(mse)

  def main_calc(new_df,main_ft):
    df_square = new_df[list(main_ft.index)]
    sqr_name = [fa+"_power_2" for fa in df_square.columns]
    log_p_name = [fa+"_log_p_one_abs" for fa in df_square.columns]
    rec_p_name = [fa+"_recip_p_one" for fa in df_square.columns]
    sqrt_name = [fa+"_sqrt_p_one" for fa in df_square.columns]

    df_sqr = pd.DataFrame(np.power(df_square.values, 2),columns=sqr_name, index=new_df.index)
    df_log = pd.DataFrame(np.log(df_square.add(1).abs().values),columns=log_p_name, index=new_df.index)
    df_rec = pd.DataFrame(np.reciprocal(df_square.add(1).values),columns=rec_p_name, index=new_df.index)
    df_sqrt = pd.DataFrame(np.sqrt(df_square.abs().add(1).values),columns=sqrt_name, index=new_df.index)

    dfs = [df_sqr, df_log, df_rec, df_sqrt]

    df_connect=  pd.concat(dfs, axis=1)

    return df_connect

  ## An attempt to decrease the amount of interactin features.

  select_ft = shap_new_abs[shap_new_abs.cumsum().sub((shap_new_abs.sum()*contribution_portion)).le(0)]
  select_ft = list(select_ft.index)
  select_ft.append(target)

  shap_select, explainer, model = shap_frame(first[select_ft], second[select_ft], target )
  shap_select_abs = shap_select[0].abs()

  ### Interactions Features
  shap_interaction_values = explainer.shap_interaction_values(first[select_ft].drop(cols_drop, axis=1))

  shap_interaction_values_abs = abs(shap_interaction_values)
  df_start = pd.DataFrame(np.sum(shap_interaction_values_abs ,axis=0),columns=first[select_ft].drop(cols_drop, axis=1).columns, index=first[select_ft].drop(cols_drop, axis=1).columns)

  #the matrix is symmetric so we need to extract upper triangle matrix without diagonal (k = 1)
  sol = (df_start.where(np.triu(np.ones(df_start.shape), k=1).astype(np.bool))
                  .stack()
                  .sort_values(ascending=False))
  #first element of sol series is the pair with the bigest correlation

  ## New Data Frames From Feature Interaction and Main Feature
  ## If you have features [a, b, c] the default polynomial features(in sklearn the degree is 2) should be [1, a, b, c, a^2, b^2, c^2, ab, bc, ca].

  ## Interaction Calculations

  dab = sol[sol.cumsum().sub((sol.sum()*inter_portion)).le(0)]

  list_one = [da[0] for da in dab.index]
  list_two = [da[1] for da in dab.index]

  def inter_cal(list_one, list_two,new_df):

    mult = [ra+"_X_"+ba for ra, ba in zip(list_one, list_two)]
    div = [ra+"_/_"+ba for ra, ba in zip(list_one, list_two)]
    print("len one " + str(len(list_one)) )
    print("len two " + str(len(list_two)) )
    inter_mult = pd.DataFrame(new_df[list_one].values*new_df[list_two].values, columns=mult, index=new_df.index)
    div_p_one = pd.DataFrame(new_df[list_one].add(1).values/new_df[list_two].add(1).values, columns=div, index=new_df.index)

    df_one = pd.concat((inter_mult,div_p_one), axis=1)

    return df_one


  def combine(target, list_one, list_two, main_ft, new_df):

    inter_mult = inter_cal(list_one, list_two, new_df.drop(columns=[target]))

    df_sqr = main_calc(new_df.drop(columns=[target]),main_ft)

    new = pd.concat((inter_mult,df_sqr),axis=1)
    new2 = pd.concat((new_df,new),axis=1)

    return new2

  new_first = combine(target, list_one, list_two, main_ft, first)

  new_second = combine(target, list_one, list_two, main_ft, second)

  if final:
    print("final")
    shap_select, explainer, model = shap_frame(new_first, new_second,target )
    ### Can be put into function, somewhat unnecessary
    shap_select_abs = shap_select[0].abs()
    shap_select_abs = shap_select_abs.sort_values(ascending=False)
    final_ft = shap_select_abs[shap_select_abs.cumsum().sub((shap_select_abs.sum()*final_contribution)).le(0)]
    final_ft = list(final_ft.index)
    final_ft.append(target)
    return new_first,new_second, mse,final_ft, shap_select_abs

  return new_first,new_second, mse

In [0]:
import gc
def runner(first_run, second_run, target, inter_portion=0.8, sqr_portion=0.8, contribution_portion=0.9, final_contribution=0.95, deflator=0.7, runs=4):
#def runner(first_run, second_run, target, inter_portion=0.4, sqr_portion=0.3, contribution_portion=0.9, deflator=0.6, runs=3):
  for r in range(runs):
    r += 1
    if r ==runs:
      print("final")
      first_run, second_run, mse, shapper, shap_select_abs = modelling(first_run, second_run, target, inter_portion, sqr_portion, contribution_portion, final_contribution, True)
    else:
      first_run, second_run, mse = modelling(first_run, second_run, target, inter_portion, sqr_portion, contribution_portion, final_contribution, False)

    inter_portion = inter_portion * deflator
    sqr_portion = sqr_portion * deflator
    contribution_portion = contribution_portion * deflator
    gc.collect()
  
  return first_run, second_run, shapper, shap_select_abs

In [57]:
new_first_two, new_second_two, shapper, shap_select_abs = runner(first, second, target)

Feature shape: (54,)


NameError: ignored

In [7]:
new_first_two, new_second_two, shapper, shap_select_abs = runner(first, second, target)

Setting feature_perturbation = "tree_path_dependent" because no background data was given.


11401.16549954644


Setting feature_perturbation = "tree_path_dependent" because no background data was given.


len one 21
len two 21
len one 21
len two 21
11668.227006723617


Setting feature_perturbation = "tree_path_dependent" because no background data was given.


len one 12
len two 12
len one 12
len two 12
11726.248034869697


Setting feature_perturbation = "tree_path_dependent" because no background data was given.


len one 4
len two 4
len one 4
len two 4
final
11810.967778923108


Setting feature_perturbation = "tree_path_dependent" because no background data was given.


len one 3
len two 3
len one 3
len two 3
final


In [8]:
new_first_two.shape

(5226, 238)

In [0]:
cap = shapper

In [0]:
# shap_select_abs = shap_select_abs.sort_values(ascending=False)
# shap_select_abs[shap_select_abs.cumsum().sub((shap_select_abs.sum()*0.9)).le(0)].shape


In [0]:
import pandas as pd
from sklearn import preprocessing

def scaler(df):
  x = df.values #returns a numpy array
  min_max_scaler = preprocessing.MinMaxScaler()
  x_scaled = min_max_scaler.fit_transform(x)
  df = pd.DataFrame(x_scaled, index=df.index, columns=df.columns)
  return df

shapper = new_first_two.columns
shapper = cap

new_first_y = new_first_two[target]
new_first = new_first_two[shapper]
new_first = scaler(new_first.drop([target],axis=1))

new_second_y = new_second_two[target]
new_second = new_second_two[shapper]
new_second = scaler(new_second.drop([target],axis=1)) 

In [50]:
from sklearn import linear_model
clf = linear_model.Lasso(alpha=0.06, tol =0.001)
lass = clf.fit(new_first,new_first_y)
preds = lass.predict(new_second)
mse = mean_squared_error(new_second_y, preds)
print(mse)

# #12381 0.10


13650.30893599866


In [0]:
from sklearn.feature_selection import SelectFromModel
# max_features
model = SelectFromModel(lass, prefit=True)
X_train = new_first_two[new_first.columns[model.get_support(indices=True)]]
new_first_y = new_first_two[target]


X_test = new_second_two[list(X_train.columns)]
new_second_y = new_second_two[target]


In [52]:
from sklearn import linear_model
lm = linear_model.LinearRegression()
lm = lm.fit(X_train.values,new_first_y.values)
preds = lm.predict(X_test.values)

mse = mean_squared_error(new_second_y.values, preds)
print(mse)

#12381 0.10

12460.55605869834


In [18]:
from sklearn import linear_model
lm = linear_model.LinearRegression()
lm = lm.fit(X_train.values,new_first_y.values)
preds = lm.predict(X_test.values)

mse = mean_squared_error(new_second_y.values, preds)
print(mse)

#12381 0.10

12460.55605869834


In [0]:
import pandas as pd
from sklearn import preprocessing

def scaler(df):
  x = df.values #returns a numpy array
  min_max_scaler = preprocessing.MinMaxScaler()
  x_scaled = min_max_scaler.fit_transform(x)
  df = pd.DataFrame(x_scaled)
  return df

add_first_y = first[target]
add_first = scaler(first.drop([target],axis=1))

add_second_y = second[target]
add_second = scaler(second.drop([target],axis=1)) 


In [22]:
from sklearn import linear_model
#clf = linear_model.Lasso(alpha=0.4)
clf = linear_model.LinearRegression()
preds = clf.fit(add_first,add_first_y).predict(add_second)
mse = mean_squared_error(add_second_y, preds)
print(mse)


15837.217936276827


In [30]:
X_train.columns

Index(['bedrooms_X_bathrooms_X_longitude', 'bedrooms_X_bathrooms_/_longitude',
       'bedrooms_X_Entire home/apt_/_longitude',
       'bedrooms_X_Entire home/apt_/_accommodates_/_latitude_X_bedrooms_X_Entire home/apt_X_bedrooms_X_bathrooms_X_accommodates_X_security_deposit_recip_p_one',
       'bathrooms_X_past_and_future_popularity', 'bathrooms_/_latitude',
       'bathrooms_X_security_deposit_recip_p_one',
       'accommodates_/_latitude_X_Entire home/apt', 'number_of_reviews',
       'bedrooms_X_Entire home/apt_/_accommodates_/_latitude_X_bedrooms_X_Entire home/apt_X_bedrooms_X_bathrooms_X_bedrooms_/_past_and_future_popularity',
       'review_scores_rating', 'cleaning_fee',
       'bedrooms_X_Entire home/apt_/_accommodates_/_latitude_X_bedrooms_X_Entire home/apt_X_bedrooms_X_bathrooms_recip_p_one',
       'bedrooms_X_Entire home/apt_/_bedrooms_X_bathrooms',
       'bathrooms_/_past_and_future_popularity_X_accommodates_X_security_deposit',
       'bathrooms_/_past_and_future_popula

In [0]:
new_first_two

In [53]:
list(new_first.columns)

['bedrooms_X_Entire home/apt_/_accommodates_/_latitude_X_bedrooms_X_Entire home/apt_X_bedrooms_X_bathrooms_X_accommodates_/_past_and_future_popularity',
 'accommodates_/_latitude_/_Entire home/apt_recip_p_one',
 'accommodates_/_latitude_/_Entire home/apt_recip_p_one',
 'bedrooms_X_bathrooms_X_longitude',
 'bedrooms_X_Entire home/apt_/_accommodates_/_latitude_recip_p_one',
 'bedrooms_X_bathrooms_/_longitude',
 'accommodates_X_past_and_future_popularity',
 'bedrooms_X_Entire home/apt_/_accommodates_/_latitude_X_bedrooms_X_Entire home/apt_X_bedrooms_X_bathrooms',
 'security_deposit_X_latitude',
 'bathrooms_per_person_/_security_deposit',
 'accommodates_/_latitude_/_Entire home/apt',
 'bedrooms_X_Entire home/apt_/_longitude',
 'accommodates_/_past_and_future_popularity_recip_p_one',
 'bedrooms_X_Entire home/apt_/_bathrooms_/_past_and_future_popularity',
 'bedrooms_X_Entire home/apt_/_accommodates_X_security_deposit',
 'bedrooms_X_bathrooms_X_longitude_X_bedrooms_X_Entire home/apt_/_accommodates_/_latitude',
 'bedrooms_X_Entire home/apt_/_accommodates_/_latitude_X_bedrooms_X_Entire home/apt_X_bedrooms_X_bathrooms_X_accommodates_X_security_deposit_recip_p_one',
 'bathrooms_X_past_and_future_popularity',
 'bedrooms_X_Entire home/apt_X_accommodates_/_latitude',
 'bedrooms_X_latitude',
 'accommodates_/_latitude_X_bathrooms_/_past_and_future_popularity',
 'bathrooms_/_latitude',
 'bathrooms_X_security_deposit_recip_p_one',
 'accommodates_/_latitude_X_Entire home/apt',
 'bedrooms_X_bathrooms_X_longitude_recip_p_one',
 'accommodates_/_latitude_/_bedrooms_/_latitude',
 'number_of_reviews',
 'bedrooms_X_Entire home/apt_/_accommodates_/_latitude_X_bedrooms_X_Entire home/apt_X_bedrooms_X_bathrooms_X_bedrooms_/_past_and_future_popularity',
 'bedrooms_X_Entire home/apt_/_accommodates_/_latitude',
 'review_scores_rating',
 'accommodates_/_past_and_future_popularity',
 'cleaning_fee',
 'bedrooms_X_Entire home/apt_/_accommodates_/_latitude_X_bedrooms_X_Entire home/apt_X_bedrooms_X_bathrooms_recip_p_one',
 'bedrooms_X_Entire home/apt_/_bedrooms_X_bathrooms',
 'bathrooms_/_past_and_future_popularity_X_accommodates_X_security_deposit',
 'bathrooms_/_past_and_future_popularity_/_accommodates_X_security_deposit',
 'bathrooms_/_past_and_future_popularity',
 'bedrooms_X_Entire home/apt_X_bathrooms_X_accommodates',
 'availability_365',
 'accommodates_/_latitude_X_bedrooms_/_latitude',
 'bedrooms_X_bathrooms_X_longitude_X_bedrooms_X_Entire home/apt_X_bedrooms_X_bathrooms',
 'security_deposit_/_past_and_future_popularity',
 'bedrooms_X_Entire home/apt_/_accommodates_/_latitude_X_accommodates_/_past_and_future_popularity',
 'bedrooms_X_bathrooms_X_longitude_/_bedrooms_X_Entire home/apt_/_accommodates_/_latitude',
 'bedrooms_/_past_and_future_popularity_recip_p_one',
 'bedrooms_X_Entire home/apt_/_accommodates_/_latitude_X_bedrooms_X_Entire home/apt_X_bedrooms_X_bathrooms_/_accommodates_X_security_deposit_recip_p_one',
 'security_deposit_/_latitude',
 'bedrooms_X_past_and_future_popularity',
 'bedrooms_/_past_and_future_popularity',
 'bedrooms_/_latitude',
 'longitude',
 'bedrooms_X_Entire home/apt_X_longitude',
 'bedrooms_X_Entire home/apt_X_accommodates_X_security_deposit',
 'Boutique hotel',
 'strict_14_with_grace_period',
 'bedrooms_/_past_and_future_popularity_power_2',
 'accommodates_X_security_deposit',
 'House',
 'latitude',
 'accommodates_X_security_deposit_recip_p_one',
 'bathrooms_X_accommodates',
 'bedrooms_X_Entire home/apt_X_bathrooms_/_past_and_future_popularity',
 'Shared room',
 'bathrooms_/_security_deposit',
 'bedrooms_X_Entire home/apt_X_bedrooms_X_bathrooms',
 'bedrooms_X_bathrooms_X_longitude_/_bedrooms_X_Entire home/apt_X_bedrooms_X_bathrooms',
 'bedrooms_X_Entire home/apt_/_accommodates_/_latitude_X_bedrooms_X_Entire home/apt_X_bedrooms_X_bathrooms_/_accommodates_/_past_and_future_popularity',
 'bathrooms_per_person_X_security_deposit',
 'bathrooms_/_accommodates',
 'bathrooms_X_latitude',
 'Private room',
 'accommodates_/_latitude_pow

In [0]:
# from sklearn.preprocessing import PolynomialFeatures
# poly_reg = PolynomialFeatures(degree=3)
# PR = poly_reg.fit(add_first)

# X_poly = PR.transform(add_first)
# X_pred = PR.transform(add_second)

# pol_reg = linear_model.LinearRegression()
# pol_reg.fit(X_poly, add_first_y)
# preds = pol_reg.predict(X_pred)
# mse = mean_squared_error(add_second_y, preds)
# print(mse)